In [1]:
pip install minsearch

Note: you may need to restart the kernel to use updated packages.


In [3]:
import minsearch

In [4]:
import json

In [7]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [9]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [11]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [13]:
index.fit(documents)

In [15]:
from openai import OpenAI

In [21]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [24]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [26]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': q}]
)

NameError: name 'q' is not defined

In [164]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [268]:
def build_propmt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.

    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [168]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response.choices[0].message.content

In [266]:
prompt = build_propmt(query, search_results)
print(len(prompt))
print(prompt)

2179
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.

    
    QUESTION: I just discovered the course. Can I still join it?
    
    CONTEXT:
    section: Module 6: streaming with kafka
question: Java Kafka: How to run producer/consumer/kstreams/etc in terminal
answer: In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java

section: Module 6: streaming with kafka
question: Module “kafka” not found when trying to run producer.py
answer: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.
To create a virtual env and install packages (run only once)
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
To activate it (you'll need to run it every time you need the virtual env):
source env/bin/activate
To deacti

In [180]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_propmt(query, search_results)
    answer = llm(prompt)
    return answer

In [182]:
rag(query)

"To run Kafka, particularly if you're working with Java, navigate to your project directory and execute the following command in the terminal:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf you're working with Python, ensure you activate a virtual environment and have all dependencies installed as per the `requirements.txt` file. If using Docker, ensure all Docker images are running.\n\nPlease note the specifics will depend on your programming language and setup."

In [184]:
rag('the course already started. can i still enroll?')

"Yes, you can still enroll in the course after it has started. You are eligible to submit the homework, but keep in mind that there are deadlines for the final project, so it's important not to leave everything until the last minute."

In [186]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [188]:
from elasticsearch import Elasticsearch

In [190]:
es_client = Elasticsearch('http://localhost:9200')

In [192]:
es_client.info()

ObjectApiResponse({'name': '8af51d8fa40d', 'cluster_name': 'docker-cluster', 'cluster_uuid': '5QhmoNWDTqWrklH1Ov4TxQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [196]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [202]:
from tqdm.auto import tqdm

In [208]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [214]:
query = 'I just discovered the course. Can I still join it?'

In [230]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [236]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_propmt(query, search_results)
    answer = llm(prompt)
    return answer

In [240]:
rag(query)

"Yes, you can still join the course even if you discovered it after the start date. You are eligible to submit the homework assignments, but be sure to keep an eye on the deadlines for the final projects so you don't leave everything for the last minute."

In [254]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'GNErYJcBObGqPdYPPhMs',
  '_score': 73.45656,
  '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
   'section': 'General course-related questions',
   'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'zNErYJcBObGqPdYPuxa4',
  '_score': 73.45656,
  '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
   'section': 'General course-related questions',
   'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 